# Optimización de performance

Evitar variables globales: ¿qué sucede con el compilador?

In [1]:
c = rand(1000,1000)
d = rand(1000,1000)

1000×1000 Array{Float64,2}:
 0.48395    0.291437   0.286854    …  0.505113   0.815323   0.444022 
 0.814194   0.842729   0.535617       0.458268   0.480075   0.801101 
 0.430665   0.59419    0.950514       0.218688   0.665771   0.715789 
 0.608295   0.660386   0.0755761      0.0825503  0.505926   0.898122 
 0.343475   0.0586935  0.177195       0.483827   0.405144   0.596541 
 0.393671   0.347296   0.878146    …  0.093587   0.727546   0.0864433
 0.380984   0.887775   0.806687       0.65902    0.290493   0.411635 
 0.259834   0.538151   0.412782       0.0494153  0.0694059  0.918574 
 0.505491   0.842588   0.461904       0.438825   0.41342    0.46653  
 0.843378   0.811042   0.914649       0.116432   0.792279   0.0358602
 0.631013   0.78837    0.685348    …  0.145372   0.0801918  0.258188 
 0.898422   0.83187    0.302495       0.334742   0.200203   0.302883 
 0.0591265  0.24688    0.0341487      0.736581   0.788352   0.0209348
 ⋮                                 ⋱                          

In [2]:
@time c*d

  0.356839 seconds (893.97 k allocations: 50.222 MiB, 2.68% gc time)


1000×1000 Array{Float64,2}:
 245.537  253.919  259.701  250.221  …  257.845  250.227  258.92   250.341
 246.337  252.388  251.196  249.677     250.484  246.335  250.265  248.419
 248.225  250.337  256.792  253.107     257.227  250.44   254.62   255.843
 248.751  253.711  254.878  247.682     256.767  238.937  258.265  249.672
 241.73   242.885  252.058  239.694     247.136  243.46   244.918  243.0  
 243.665  255.169  254.054  253.081  …  253.716  252.06   253.441  250.089
 237.445  245.828  243.69   242.404     243.498  244.181  245.478  240.819
 252.171  251.047  262.922  256.653     254.886  250.076  255.933  253.587
 240.321  251.051  251.62   239.43      247.968  240.515  246.164  244.846
 243.585  248.144  250.348  247.135     253.315  245.169  253.202  249.749
 241.714  252.19   245.176  243.279  …  244.006  244.367  251.135  242.821
 243.25   242.467  253.374  246.878     249.28   247.469  251.748  243.263
 241.328  243.16   252.514  247.772     246.095  242.572  252.95   244.2

In [3]:
function mult_n(n)
    a = rand(n, n)
    b = rand(n, n)
        return a*b
end

mult_n (generic function with 1 method)

In [5]:
@time mult_n(1000)

  0.046333 seconds (6.09 k allocations: 23.160 MiB, 8.40% gc time)


1000×1000 Array{Float64,2}:
 257.166  258.969  251.529  266.113  …  248.664  248.9    253.767  257.958
 261.955  261.226  244.753  270.476     255.945  257.095  259.908  264.282
 252.301  254.635  244.503  253.994     244.132  241.645  247.399  254.454
 253.533  241.684  240.572  253.577     238.002  243.699  246.616  251.758
 257.865  256.562  246.854  258.545     245.37   249.394  249.303  257.104
 255.359  254.027  248.896  257.96   …  242.748  243.818  254.731  259.097
 253.724  257.997  245.755  262.168     244.589  244.226  251.105  257.423
 258.25   260.364  248.225  269.821     247.026  257.437  256.537  256.564
 254.336  250.624  241.805  251.641     241.221  246.535  242.962  255.108
 252.622  257.91   244.274  253.141     241.318  243.055  248.985  250.241
 250.329  244.47   240.144  251.799  …  239.875  242.682  247.165  251.137
 250.091  248.303  240.744  255.194     240.55   242.432  243.896  251.516
 241.644  250.99   238.407  258.235     241.939  244.151  240.64   253.5

Evitar campos con tipos abstractos: ¿qué usa el compilador para compilar?

In [6]:
struct MiTipoAmbiguo
    dato
end

In [7]:
instancia1 = MiTipoAmbiguo(3)
instancia2 = MiTipoAmbiguo("Hola")

MiTipoAmbiguo("Hola")

In [8]:
typeof(instancia1)

MiTipoAmbiguo

In [9]:
typeof(instancia2)

MiTipoAmbiguo

In [10]:
struct MiTipo{T<:AbstractFloat}
    a::T
end

In [11]:
instancia3 = MiTipo(3.0)
typeof(instancia3)

MiTipo{Float64}

Escribir funciones con tipos estables.

In [15]:
fixPos(x) = x < 0 ? zero(x) : x

fixPos (generic function with 1 method)

In [41]:
typeof(fixPos(12.3))

Float64

In [42]:
typeof(fixPos(-4.0))

Float64

Evitar cambiar el tipo de una variable dentro de una función

In [32]:
function foo()
    x = 1
    for i = 1:10
        x /= rand()
    end
    return x
end

foo (generic function with 1 method)

In [33]:
@time foo()

  0.023822 seconds (23.55 k allocations: 1.244 MiB)


246574.19711758065

In [34]:
function foo2()
    x::Float64 = 1 / rand()
    for i = 2:10
        x /= rand()
    end
    return x
end

foo2 (generic function with 1 method)

In [35]:
@time foo2()

  0.016112 seconds (25.89 k allocations: 1.349 MiB)


9467.162482867509